In [1]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import torch
from torch import nn

import flatiron.torch.tools as fi_torchtools
from flatiron.core.dataset import Dataset
from flatiron.torch.tools_test import SimpleModel
import cv_depot.api as cvd

2025-02-18 02:25:29.576723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739845529.593245  104060 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739845529.598290  104060 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 02:25:29.615743: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import flatiron
opt = flatiron.torch.optimizer.get('SGD')

torch.optim.sgd.SGD

In [2]:
model = SimpleModel(2, 1, 2)
compiled = fi_torchtools.compile(
    model=model, optimizer='Adam', loss='CrossEntropyLoss',
    metrics=['Accuracy'], device='cuda', kwargs=dict(mode='reduce-overhead')
)

TypeError: Adam.__init__() missing 1 required positional argument: 'params'

In [3]:
src = '/mnt/storage/projects/unet001/dset002/p-unet001_s-dset002_d-glom_v001'
data = Dataset.read_directory(src, labels=['a'])
train, test = data.train_test_split()
# train = fi_torchtools.TorchDataset.monkey_patch(train)
# test = fi_torchtools.TorchDataset.monkey_patch(test)

In [9]:
compiled

{'model': OptimizedModule(
   (_orig_mod): SimpleModel(
     (layer_stack): Sequential(
       (0): Flatten(start_dim=1, end_dim=-1)
       (1): Linear(in_features=2, out_features=1, bias=True)
       (2): Linear(in_features=1, out_features=2, bias=True)
     )
   )
 ),
 'optimizer': torch.optim.adam.Adam,
 'loss': torch.nn.modules.loss.CrossEntropyLoss,
 'metrics': [torchmetrics.classification.accuracy.Accuracy],
 'device': 'cuda'}

In [8]:
fi_torchtools.train(
    compiled,
    [],
    train,
    test,
    batch_size=10,
    seed=42,
)

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: Optimizer.zero_grad() missing 1 required positional argument: 'self'

In [ ]:
src = '/mnt/storage/projects/unet001/dset001/p-unet001_s-dset001_d-glom_v001'
data = Dataset.read_directory(src).load(reshape=False).data

root = '/mnt/storage/projects/unet001/dset002/p-unet001_s-dset002_d-glom_v001'
info = pd.DataFrame()
info['frame'] = list(range(100))
info['asset_path'] = root
info['filepath_relative'] = info.frame.apply(lambda x: f'data/p-unet001_s-dset002_d-glom_v001_f{x:02d}.exr')
info['filepath'] = info.apply(lambda x: Path(x.asset_path, x.filepath_relative), axis=1)
info['content'] = info.frame.apply(lambda x: cvd.Image.from_array(data[x]))

tgt = Path(root, 'data')
os.makedirs(tgt, exist_ok=True)
info.apply(lambda x: x.content.write(x.filepath), axis=1)

del info['frame']
del info['content']
del info['filepath']
tgt = Path(root, 'p-unet001_s-dset002_d-glom_v001.csv')
info.to_csv(tgt, index=None)